In [45]:
import re
import sys
import nltk
import json
import warnings
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

warnings.filterwarnings('ignore')
np.set_printoptions(threshold=sys.maxsize)

In [46]:
# Load all the data
trainpath = 'data/train.jsonl'
testpath = 'data/test.jsonl'
valpath = 'data/validation.jsonl'

traindata = pd.read_json(trainpath, lines=True)
testdata = pd.read_json(testpath, lines=True)
valdata = pd.read_json(valpath, lines=True)

traindata = traindata[['sentence1', 'sentence2', 'score']]
testdata = testdata[['sentence1', 'sentence2', 'score']]
valdata = valdata[['sentence1', 'sentence2', 'score']]

In [47]:
def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', text)

def remove_number(text):
    return re.sub(r'\d+', 'num', text)

def replace_url(text):
    return re.sub(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', 'url', text)

def replace_hashtags(text):
    return re.sub(r'#[a-zA-Z\d]+', 'hashtag', text)

def replace_email(text):
    return re.sub(r'[a-zA-Z\.]+@[a-zA-Z\.\d]+', 'email', text)

def replace_mentions(text):
    return re.sub(r'@[a-zA-Z\.\d_]+', 'mention', text)

In [48]:
# Stop words, Lemmatization and Stemming
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

In [49]:
def preprocess_text(text):
    text = remove_punctuation(text)
    text = remove_number(text)
    text = replace_url(text)
    text = replace_hashtags(text)
    text = replace_email(text)
    text = replace_mentions(text)
    # convert to lower case
    text = text.lower()
    sentence = text.split()
    # remove stop words
    sentence = [word for word in sentence if word not in stop_words]
    # apply lemmatize
    sentence = [lemmatizer.lemmatize(word) for word in sentence]
    # apply stemming
    sentence = [stemmer.stem(word) for word in sentence]
    return sentence

def process_data(data):
    data['sentence1'] = data['sentence1'].apply(lambda x: preprocess_text(x))
    data['sentence2'] = data['sentence2'].apply(lambda x: preprocess_text(x))
    return data
    
def get_vocab(data):
    vocab = set()
    for _, row in data.iterrows():
        for word in row['sentence1']:
            vocab.add(word)
        for word in row['sentence2']:
            vocab.add(word)
    return vocab

In [50]:
traindata = process_data(traindata)
testdata = process_data(testdata)
valdata = process_data(valdata)

# Create vocab and word2idx
vocab = get_vocab(traindata)
word2idx = {word: idx for idx, word in enumerate(sorted(vocab))}

file_path = 'word2idx.json'
file_path = 'data/word2idx.json'

# # Save the dictionary to a JSON file
with open(file_path, 'w') as file:
    json.dump(word2idx, file)

In [51]:
# Store the processed datasets
traindata.to_csv('data/train.csv', index=False)
testdata.to_csv('data/test.csv', index=False)
valdata.to_csv('data/validation.csv', index=False)

In [52]:
print(len(vocab))

8258


In [53]:
print(traindata[:5])

                             sentence1  \
0                        [plane, take]   
1             [man, play, larg, flute]   
2  [man, spread, shrede, chees, pizza]   
3            [three, men, play, chess]   
4                   [man, play, cello]   

                                    sentence2  score  
0                          [air, plane, take]   5.00  
1                          [man, play, flute]   3.80  
2  [man, spread, shred, chees, uncook, pizza]   3.80  
3                     [two, men, play, chess]   2.60  
4                    [man, seat, play, cello]   4.25  
